## 1.Data extraction and processing

In [ ]:
# Import required modules
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd

In [ ]:
nameList=['','1.Xiaoming','2.Lihua','3.Lilei','4.Liping','5.Zhangli','6.Zhangsong','7.Pingan','8.Wuhan','9.Liusong','10.Wuyong','11.Donald Trump','12.Joseph Robinette Biden,Jr','13.ntony Blinken','14.Lloyd Austin','15.Hillary Diane Rodham Clinton','16.Niyuwei','17.Zhanghongjun','18.Lionel Messi','19.Cristiano Ronaldo','20.Liyifeng','21.Angelina','22.Anna','23.Macron','24.Rishi','25.Scarlett','26.Zhang Hongjun','27.Wu Jingk','28.Shen Teng','29.Sun Li','30.Deng Chao','31.Chenqiaoen','32.Yangying,mask','33.Sunhonglei','34.Hanxue','35.Jingboran','36.Dilireba','37.Yangmi','38.Zhangxinyu','39.Niyuwei','40.Jane','41.Yan Shirui','42.Yan Zhihou']

In [ ]:
# plt displays grayscale images
def plt_show(img):
    plt.imshow(img,cmap='gray')
    plt.show()

In [ ]:
# Read all images in a folder, input parameter is file name, return file address list
def read_directory(directory_name):
    faces_addr = []
    for filename in os.listdir(directory_name):
        faces_addr.append(directory_name + "/" + filename)
    return faces_addr

In [ ]:
# Read all face folders and save image addresses in the list
faces = []
for i in range(1,43):###########
    faces_addr = read_directory('./train_f/s'+str(i))
    for addr in faces_addr:
        faces.append(addr)

In [ ]:
# Read the image data and generate the list label
images = []
labels = []
for index,face in enumerate(faces):
    image = cv2.imread(face,0)
    images.append(image)
    labels.append(int(index/5+1))
print(len(labels))
print(len(images))
print(type(images[0]))
print(labels)

## 2. PCA dimension reduction

In [ ]:
# Transform the image data into a eigenmatrix
image_data = []
for image in images:
    data = image.flatten()
    image_data.append(data)
print(image_data[0].shape)

In [ ]:
# Convert to a numpy array
X = np.array(image_data)
y = np.array(labels)
print(type(X))
print(X.shape)

In [ ]:
# Import sklearn's pca module
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
# Draw the eigenmatrix
import pandas as pd
data = pd.DataFrame(X)
data.head()

In [ ]:
# Partition data set
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Training PCA model
pca=PCA(n_components=160)
pca.fit(x_train)

In [ ]:
# Returns the data set after dimensionality reduction of the test set and training set
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)
print(x_train_pca.shape)
print(x_test_pca.shape)

In [ ]:
V = pca.components_
V.shape

In [ ]:
# Feature face
# Create the canvas and subgraph objects
fig, axes = plt.subplots(4,40
                       ,figsize=(15,15)
                       ,subplot_kw = {"xticks":[],"yticks":[]} #Do not display axes
                       )
#Fill image
for i, ax in enumerate(axes.flat):
    ax.imshow(V[i,:].reshape(100,100),cmap="gray") #Select a color pattern

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# Returns how much of the original data the feature carries
pca.explained_variance_ratio_.sum()

In [ ]:
# Plot the number of features and the number of messages they carry
explained_variance_ratio = []
for i in range(1,160): 
    pca=PCA(n_components=i).fit(x_train)
    explained_variance_ratio.append(pca.explained_variance_ratio_.sum())
plt.plot(range(1,160),explained_variance_ratio)
plt.show()

In [ ]:
# Model creation and training
model = cv2.face.EigenFaceRecognizer_create()
model.train(x_train,y_train)

In [ ]:
# Forecast
res = model.predict(x_test[0])
print(res)

In [ ]:
y_test[0]

In [ ]:
# Test the accuracy of the data set
ress = []
true = 0
for i in range(len(y_test)):
    res = model.predict(x_test[i])
#     print(res[0])
    if y_test[i] == res[0]:
        true = true+1
    else:
        print(i)

print('Test set identification accuracy:%.2f'% (true/len(y_test)))

In [ ]:
# Average face
mean = model.getMean()
print(mean)
meanFace = mean.reshape(100,100)
plt_show(meanFace)

## 3. Custom picture Test (Face recognition)

In [ ]:
# Dimension reduction
pca=PCA(n_components=160)
pca.fit(X)
X = pca.transform(X)

In [ ]:
# Use all the data as a training set
# Model creation and training
model = cv2.face.EigenFaceRecognizer_create()
model.train(X,y)

In [ ]:
# plt displays color pictures
def plt_show0(img):
    b,g,r = cv2.split(img)
    img = cv2.merge([r, g, b])
    plt.imshow(img)
    plt.show()

In [ ]:
##Face detection
def face_detection(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_detect = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    face = face_detect.detectMultiScale(gray)
    for x,y,w,h in face:
        cv2.rectangle(img,(x,y),(x+w,y+h),color=(0,0,255),thickness=2)
    cv2.imshow('result (press Esc to terminate)',img)

#Read camera
cap = cv2.VideoCapture(0)
cap.read()
while True:
    flag,frame = cap.read()
    if not flag:
        break;
    face_detection(frame)
    if cv2.waitKey(2)==27:
        break
cv2.destroyAllWindows()
cap.release()
##Save a photo
def get_photo():
    cap = cv2.VideoCapture(0)
    f, frame = cap.read()
    cv2.imwrite('./test_f/image.jpg', frame)
    cap.release()
get_photo()
###
face_engine = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
img = cv2.imread('./test_f/image.jpg')
plt_show0(img)
# Copy image grayscale processing
img_ = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
# Detect the face to get the face region
faces = face_engine.detectMultiScale(gray)
# Visualize the detected faces
for(x, y, w, h) in faces:
    cv2.rectangle(img_, (x, y), (x + w, y + h), (0, 0, 255), 3)
    plt_show0(img_)
    face = img[y:y + w + 10, x:x + h + 10]
    plt_show0(face)

In [ ]:
testImage="./test_f/image.jpg"

In [ ]:
# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the image
img = cv2.imread(testImage)
plt_show0(img)
# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect the face in the image
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

# Loop over all the faces in the image
for (x, y, w, h) in faces:
    # Add a margin around the face
    margin = 10
    x = x - margin
    y = y - margin
    w = w + margin*2
    h = h + margin*2

    # Crop the image to the face
    face_img = img[y:y+h, x:x+w]

    # Resize the face image to 100x100 pixels
    resized_face = cv2.resize(face_img, (100, 100))
plt_show0(resized_face)

In [ ]:
img = cv2.cvtColor(resized_face, cv2.COLOR_BGR2GRAY)
plt_show(img)

In [ ]:
imgs = []
imgs.append(img)

In [ ]:
# Eigenmatrix
image_data = []
for img in imgs:
    data = img.flatten()
    image_data.append(data)

In [ ]:
test = np.array(image_data)
test.shape

In [ ]:
# The trained pca model was used to reduce the dimension of the picture
test = pca.transform(test)
test[0].shape

In [ ]:
res = model.predict(test)

In [ ]:
res

In [ ]:
##Expression recognition
from deepface import DeepFace
facial=cv2.imread(testImage)
result=DeepFace.analyze(img_path=testImage,actions=['emotion'],enforce_detection=False)
plt_show0(facial)
print("Expression:")
print(result)
##Face recognition
print('Face recognition result：',res[0])
print(nameList[res[0]])